In [90]:
import pandas as pd
import xgboost as xgb
from rdkit import Chem
from rdkit.Chem import Descriptors
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from sklearn import neural_network as nn
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, KFold
import numpy as np
import matplotlib.pyplot as plt

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [91]:
df = pd.read_csv("Permuted Data")
df

,Standardized_SMILES,pchembl_value,MolWt,LogP,QED
0,Brc1cncc(COCC2(c3ccccc3)CCNCC2)c1,4.82,361.283,3.6821,0.882288
1,C1=NC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCCCC...,6.70,397.526,6.4497,0.416926
2,C1=NC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCCCC...,6.47,398.514,4.8690,0.460447
3,C1=NC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCOCC...,5.39,399.498,5.2960,0.490800
4,C1=NCCC(c2c[nH]c(-c3cc4c([nH]c5ccccc54)c(C4CCC...,7.05,397.526,6.1021,0.438245
...,...,...,...,...,...
1257,c1ccc2c3c([nH]c2c1)C(C1CCOCC1)NC(c1nc(-c2cscn2...,5.36,405.527,4.3692,0.472923
1258,c1ccc2c3c([nH]c2c1)C(C1CCOCC1)NC(c1nc(C24CC5CC...,6.30,456.634,5.7135,0.472659
1259,c1ccc2c3c([nH]c2c1)C(C1CCOCC1)NC(c1nc(C2CCCCC2...,6.87,404.558,5.2934,0.555363
1260,c1cncc(-c2c[nH]c(C3Cc4c([nH]c5ccccc45)C(C4CCCC...,6.72,397.526,5.4614,0.429771


In [105]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from math import sqrt


# Create a DeepChem dataset
tasks = ["pchembl_value"]
featurizer = dc.feat.CircularFingerprint(size=2048)
loader = dc.data.CSVLoader(tasks=tasks, feature_field="Standardized_SMILES", featurizer=featurizer)
dataset = loader.create_dataset("Permuted Data")

# Define the scaffold splitter
splitter = dc.splits.ScaffoldSplitter()

# Get scaffold split indices
train_valid_test = splitter.k_fold_split(dataset, k=10, seed=42)


In [107]:
dc.__version__

'2.7.1'

In [93]:
#MLP

import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = nn.MLPRegressor(hidden_layer_sizes=(min(25,round(np.sqrt(X_train.shape[0])),)), random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1136, Test size: 126


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 1 Accuracy: (0.5912916291602646, 3.108012849419772e-13)
Fold 1 rmse: 1.3177337378299285
Fold 1 r_squared: 0.2009916134787978
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.6965963009929846
Fold 2 - Train size: 1136, Test size: 126


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 2 Accuracy: (0.4270611658158794, 6.133761952803951e-07)
Fold 2 rmse: 0.7855805654601232
Fold 2 r_squared: -1.294977538081445
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.5051484190689668
Fold 3 - Train size: 1136, Test size: 126


...

Fold 8 Accuracy: (0.35531703587337904, 4.4511590265366995e-05)
Fold 8 rmse: 0.7924700302692059
Fold 8 r_squared: -0.3180654321178251
Fold 8 Mean of Two Highest Tanimoto Similarities: 0.6285783087909376
Fold 9 - Train size: 1135, Test size: 127


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 9 Accuracy: (0.6738880109080592, 3.875541352896469e-18)
Fold 9 rmse: 0.9659219363521504
Fold 9 r_squared: 0.4344106569667415
Fold 9 Mean of Two Highest Tanimoto Similarities: 0.6933067802988767
Fold 10 - Train size: 1135, Test size: 127


/home/jupyter-seal@broadinstitut-41f01/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Fold 10 Accuracy: (0.4239686263687994, 6.800210000953719e-07)
Fold 10 rmse: 0.8295791995603657
Fold 10 r_squared: -0.26753472428592695
Fold 10 Mean of Two Highest Tanimoto Similarities: 0.6928868649027429
Mean Pearson R: 0.30570662312439534
Overall Mean of Two Highest Tanimoto Similarities: 0.6708981912991867


In [94]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("scaffold_cv_results_MLP_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,CC(c1ccccc1Oc1ccc(Cl)c(Cl)c1)N(C)C,6.05,6.073872
1,1,CCc1cc(Oc2cc(Cl)ccc2CNC)ccc1Cl,5.94,5.933047
2,1,CCc1cc(Oc2ccc(F)cc2CNC)ccc1Cl,5.34,5.420175
3,1,CN(C)Cc1ccc(C(F)(F)F)cc1Oc1ccc(Cl)c(Cl)c1,5.95,5.952821
4,1,CN(C)Cc1ccccc1Oc1ccc(Cl)c(Cl)c1,5.67,5.759415
...,...,...,...,...
11353,10,FC(F)(F)c1cc(Cl)cc(COCC2(c3ccccc3)CCNCC2)c1,5.14,5.080808
11354,10,Cn1nc(C(=O)N2CCC(Oc3ccc(CN4CCCC4)cc3)CC2)cc1-c...,5.13,5.039410
11355,10,Cn1cc(C2(c3nc(=O)o[nH]3)NC(c3nc(-c4ccc(F)cn4)c...,5.92,5.924062
11356,10,Cn1cc(C2(c3ccc(C(=O)O)cn3)NC(c3nc(-c4ccc(F)cc4...,4.02,4.157273


In [95]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("scaffold_cv_results_MLP_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,CC(C)(O)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n5...,5.16,6.150951
1,1,CC(C)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n5)NC...,5.46,5.519859
2,1,CCOC(=O)C(CC12CCC(NCc3ccc4c(n3)NC(=O)CO4)(CC1)...,5.08,5.172382
3,1,CCOC(=O)C(F)(F)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5c...,4.59,6.167730
4,1,CCOC(=O)CCCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n...,6.00,6.172857
...,...,...,...,...
1257,10,Cn1c(SCCCN2CCC3CC3(c3ccc(C(F)(F)F)cc3)CC2)nnc1...,5.30,5.866385
1258,10,Cn1c(SCCCN2CCc3ccc4oc(C(F)(F)F)nc4c3CC2)nnc1-c...,6.60,6.382249
1259,10,Cn1cc(-c2ccc(OC3CCN(C(=O)Cc4ccc(OC(F)(F)F)cc4)...,5.17,4.719193
1260,10,Cn1cc(-c2ccc(OC3CCN(C(=O)Cc4ccc(OC(F)(F)F)cc4)...,5.35,4.806854


In [96]:
#RF

In [97]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = RandomForestRegressor(n_estimators=min(25,round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1136, Test size: 126
Fold 1 Accuracy: (0.2740705474462609, 0.0018998208492631112)
Fold 1 rmse: 1.631888990641759
Fold 1 r_squared: -0.22539818351348395
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.6965963009929846
Fold 2 - Train size: 1136, Test size: 126
Fold 2 Accuracy: (0.4781357786961515, 1.4952306054781706e-08)
Fold 2 rmse: 0.4765670278811703
Fold 2 r_squared: 0.15541189465338368
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.5051484190689668
Fold 3 - Train size: 1136, Test size: 126
Fold 3 Accuracy: (0.8480792863359191, 5.2663828973919275e-36)
Fold 3 rmse: 0.578919899605058
Fold 3 r_squared: 0.6996275758403181
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.6851938767826621
Fold 4 - Train size: 1136, Test size: 126
Fold 4 Accuracy: (0.8066745931801166, 4.190772504375468e-30)
Fold 4 rmse: 0.60822734105982
Fold 4 r_squared: 0.632389685218248
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.7029900531883766
Fold 5 - Train size: 1136, Test size

In [98]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("scaffold_cv_results_RF_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,CC(c1ccccc1Oc1ccc(Cl)c(Cl)c1)N(C)C,6.05,5.8972
1,1,CCc1cc(Oc2cc(Cl)ccc2CNC)ccc1Cl,5.94,5.8768
2,1,CCc1cc(Oc2ccc(F)cc2CNC)ccc1Cl,5.34,5.3500
3,1,CN(C)Cc1ccc(C(F)(F)F)cc1Oc1ccc(Cl)c(Cl)c1,5.95,5.9076
4,1,CN(C)Cc1ccccc1Oc1ccc(Cl)c(Cl)c1,5.67,5.6136
...,...,...,...,...
11353,10,FC(F)(F)c1cc(Cl)cc(COCC2(c3ccccc3)CCNCC2)c1,5.14,5.1608
11354,10,Cn1nc(C(=O)N2CCC(Oc3ccc(CN4CCCC4)cc3)CC2)cc1-c...,5.13,5.1020
11355,10,Cn1cc(C2(c3nc(=O)o[nH]3)NC(c3nc(-c4ccc(F)cn4)c...,5.92,5.7198
11356,10,Cn1cc(C2(c3ccc(C(=O)O)cn3)NC(c3nc(-c4ccc(F)cc4...,4.02,4.2728


In [99]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("scaffold_cv_results_RF_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,CC(C)(O)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n5...,5.16,5.4262
1,1,CC(C)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n5)NC...,5.46,5.2590
2,1,CCOC(=O)C(CC12CCC(NCc3ccc4c(n3)NC(=O)CO4)(CC1)...,5.08,5.6268
3,1,CCOC(=O)C(F)(F)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5c...,4.59,5.4488
4,1,CCOC(=O)CCCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n...,6.00,6.5384
...,...,...,...,...
1257,10,Cn1c(SCCCN2CCC3CC3(c3ccc(C(F)(F)F)cc3)CC2)nnc1...,5.30,5.9700
1258,10,Cn1c(SCCCN2CCc3ccc4oc(C(F)(F)F)nc4c3CC2)nnc1-c...,6.60,5.7000
1259,10,Cn1cc(-c2ccc(OC3CCN(C(=O)Cc4ccc(OC(F)(F)F)cc4)...,5.17,5.2496
1260,10,Cn1cc(-c2ccc(OC3CCN(C(=O)Cc4ccc(OC(F)(F)F)cc4)...,5.35,4.9296


In [100]:
#xgboost

In [101]:
import deepchem as dc
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error, r2_score

# Lists to store the results
accuracies = []
mean_nn_similarities = []
train_results = []
test_results = []

# Helper function to compute bulk Tanimoto similarity
def compute_bulk_tanimoto_similarity(train_mols, test_mols):
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in train_mols]
    test_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2) for mol in test_mols]
    tanimoto_sim = []
    for test_fp in test_fps:
        sims = DataStructs.BulkTanimotoSimilarity(test_fp, train_fps)
        tanimoto_sim.append(sims)
    return np.array(tanimoto_sim)

# Perform 10-Fold Cross Validation
for fold, (train_dataset, test_dataset) in enumerate(train_valid_test):
    # Extract the features and labels from the DeepChem datasets
    X_train, y_train = train_dataset.X, np.squeeze(train_dataset.y)
    X_test, y_test = test_dataset.X, np.squeeze(test_dataset.y)
    
    print(f"Fold {fold + 1} - Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")
    
    # Define the model
    model = xgb.XGBRegressor(n_estimators=min(25, round(np.sqrt(X_train.shape[0]))), random_state=42, n_jobs=-1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = stats.pearsonr(y_test, y_pred)
    accuracies.append(accuracy)
    
    print(f"Fold {fold + 1} Accuracy: {accuracy}")
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_squared = r2_score(y_test, y_pred)
    print(f"Fold {fold + 1} rmse: {rmse}")
    print(f"Fold {fold + 1} r_squared: {r_squared}")

    # Calculate bulk Tanimoto similarity
    train_smiles = train_dataset.ids
    test_smiles = test_dataset.ids
    
    train_mols = [Chem.MolFromSmiles(s) for s in train_smiles]
    test_mols = [Chem.MolFromSmiles(s) for s in test_smiles]
    
    tanimoto_sim = compute_bulk_tanimoto_similarity(train_mols, test_mols)
    
    # Calculate the mean of the two highest Tanimoto similarities for each test molecule
    two_highest_similarities = np.sort(tanimoto_sim, axis=1)[:, -2:]
    mean_two_highest_similarities = np.mean(two_highest_similarities, axis=1)
    mean_nn_similarities.append(np.mean(mean_two_highest_similarities))

    print(f"Fold {fold + 1} Mean of Two Highest Tanimoto Similarities: {np.mean(mean_two_highest_similarities)}")
    
    training_preds = model.predict(X_train)
    
    # Store the results
    train_fold_results = pd.DataFrame({
        "training batch": fold + 1,
        "Standardized_SMILES": train_smiles,
        "True Values": y_train,
        "Predictions": training_preds
    })
    train_results.append(train_fold_results)
    
    
    # Store the results
    test_fold_results = pd.DataFrame({
        "testing batch": fold + 1,
        "Standardized_SMILES": test_smiles,
        "True Values": y_test,
        "Predictions": y_pred
    })
    test_results.append(test_fold_results)

# Print the mean accuracy
print(f"Mean Pearson R: {np.mean(accuracies)}")

# Print the overall mean of the two highest Tanimoto Similarities
print(f"Overall Mean of Two Highest Tanimoto Similarities: {np.mean(mean_nn_similarities)}")

Fold 1 - Train size: 1136, Test size: 126
Fold 1 Accuracy: (0.39289543520303355, 5.347703411761081e-06)
Fold 1 rmse: 1.6008340119539686
Fold 1 r_squared: -0.179203102167506
Fold 1 Mean of Two Highest Tanimoto Similarities: 0.6965963009929846
Fold 2 - Train size: 1136, Test size: 126
Fold 2 Accuracy: (0.5538857892979693, 1.72623967068404e-11)
Fold 2 rmse: 0.457026128401407
Fold 2 r_squared: 0.2232539798077069
Fold 2 Mean of Two Highest Tanimoto Similarities: 0.5051484190689668
Fold 3 - Train size: 1136, Test size: 126
Fold 3 Accuracy: (0.8160256114247204, 2.639100814692282e-31)
Fold 3 rmse: 0.6370367794721868
Fold 3 r_squared: 0.6362926164990936
Fold 3 Mean of Two Highest Tanimoto Similarities: 0.6851938767826621
Fold 4 - Train size: 1136, Test size: 126
Fold 4 Accuracy: (0.7824573404437545, 2.8162947556407573e-27)
Fold 4 rmse: 0.6479570987939739
Fold 4 r_squared: 0.582796141982203
Fold 4 Mean of Two Highest Tanimoto Similarities: 0.7029900531883766
Fold 5 - Train size: 1136, Test size:

In [102]:
# Save the results to a CSV file
train_results_df = pd.concat(train_results, ignore_index=True)
train_results_df.to_csv("scaffold_cv_results_xgboost_training batches.csv", index=False)
train_results_df

,training batch,Standardized_SMILES,True Values,Predictions
0,1,CC(c1ccccc1Oc1ccc(Cl)c(Cl)c1)N(C)C,6.05,5.630945
1,1,CCc1cc(Oc2cc(Cl)ccc2CNC)ccc1Cl,5.94,5.830216
2,1,CCc1cc(Oc2ccc(F)cc2CNC)ccc1Cl,5.34,5.449155
3,1,CN(C)Cc1ccc(C(F)(F)F)cc1Oc1ccc(Cl)c(Cl)c1,5.95,5.876772
4,1,CN(C)Cc1ccccc1Oc1ccc(Cl)c(Cl)c1,5.67,5.577087
...,...,...,...,...
11353,10,FC(F)(F)c1cc(Cl)cc(COCC2(c3ccccc3)CCNCC2)c1,5.14,5.044445
11354,10,Cn1nc(C(=O)N2CCC(Oc3ccc(CN4CCCC4)cc3)CC2)cc1-c...,5.13,5.216791
11355,10,Cn1cc(C2(c3nc(=O)o[nH]3)NC(c3nc(-c4ccc(F)cn4)c...,5.92,5.306965
11356,10,Cn1cc(C2(c3ccc(C(=O)O)cn3)NC(c3nc(-c4ccc(F)cc4...,4.02,4.491025


In [103]:
# Save the results to a CSV file
test_results_df = pd.concat(test_results, ignore_index=True)
test_results_df.to_csv("scaffold_cv_results_xgboost_testing batches.csv", index=False)
test_results_df

,testing batch,Standardized_SMILES,True Values,Predictions
0,1,CC(C)(O)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n5...,5.16,5.250528
1,1,CC(C)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n5)NC...,5.46,5.515674
2,1,CCOC(=O)C(CC12CCC(NCc3ccc4c(n3)NC(=O)CO4)(CC1)...,5.08,5.144951
3,1,CCOC(=O)C(F)(F)CCOc1ccc2ncc(F)c(CCC34CCC(NCc5c...,4.59,5.030062
4,1,CCOC(=O)CCCOc1ccc2ncc(F)c(CCC34CCC(NCc5ccc6c(n...,6.00,5.841728
...,...,...,...,...
1257,10,Cn1c(SCCCN2CCC3CC3(c3ccc(C(F)(F)F)cc3)CC2)nnc1...,5.30,5.844471
1258,10,Cn1c(SCCCN2CCc3ccc4oc(C(F)(F)F)nc4c3CC2)nnc1-c...,6.60,5.799333
1259,10,Cn1cc(-c2ccc(OC3CCN(C(=O)Cc4ccc(OC(F)(F)F)cc4)...,5.17,5.061011
1260,10,Cn1cc(-c2ccc(OC3CCN(C(=O)Cc4ccc(OC(F)(F)F)cc4)...,5.35,5.019034
